# Wjet Signal Region Shape Extraction Using Alpha Ratio Method

1. Calculate alpha using MC only
    1. Ratio of signal to side-band region
1. Get 1-sigma up/down alpha shape by varying the fit parameters by 1 sigma.
1. Grab corrected W+jet shape and normalization in side-band region from background estimation code.
1. Using alpha get the W+jet shape and normalization in signal region
1. Also, grab the corrected W+jet shape and its up/down from wjet fit parameters.

In [1]:
import ROOT as r
import math

%jsroot on

# To surpress unwanted messages
msgservice = r.RooMsgService.instance()
msgservice.setGlobalKillBelow(r.RooFit.FATAL)

import CMS_lumi, tdrstyle

tdrstyle.setTDRStyle()
r.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.08/05


In [2]:
#fin = r.TFile("Ntuples2/WWTree_VJets.root","read")
fin = r.TFile("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_05_15_04h15/HaddedFiles/Hadds_for_BkgEstimation/WWTree_VJets.root","read")


treeIn = fin.Get("otree")

# Define the cuts
cutString_SideBand       =  "wSampleWeight*35867.06*btag0Wgt*genWeight*trig_eff_Weight*id_eff_Weight*pu_Weight*((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && (((PuppiAK8_jet_mass_so_corr>40)&&(PuppiAK8_jet_mass_so_corr<65)) || ((PuppiAK8_jet_mass_so_corr>105)&&(PuppiAK8_jet_mass_so_corr<150))) && (mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

cutString_Signal         =  "wSampleWeight*35867.06*btag0Wgt*genWeight*trig_eff_Weight*id_eff_Weight*pu_Weight*((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && ((PuppiAK8_jet_mass_so_corr>65) &&(PuppiAK8_jet_mass_so_corr<105)) && (mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

## Get W+jet histogram in signal, as well as side band region

In [3]:
hSignal_4bin = r.TH1D("hSignal_4bin","",4,600,5000)
hSignal_4bin.SetMarkerColor(1)
hSignal_4bin.SetLineColor(1)
hSignal_4bin.SetLineWidth(3)
hSignal_4bin.SetMarkerSize(1)

hSignal_15bin = r.TH1D("hSignal_15bin","",15,600,5000)
hSignal_15bin.SetMarkerColor(1)
hSignal_15bin.SetLineColor(1)
hSignal_15bin.SetLineWidth(3)
hSignal_15bin.SetMarkerSize(1)

hSignal_88bin = r.TH1D("hSignal_88bin","",88,600,5000)
hSignal_88bin.SetMarkerColor(1)
hSignal_88bin.SetLineColor(1)
hSignal_88bin.SetLineWidth(3)
hSignal_88bin.SetMarkerSize(1)

hSideBand_15bin = r.TH1D("hSideBand_15bin","",15,600,5000)
hSideBand_15bin.SetMarkerColor(2)
hSideBand_15bin.SetLineColor(2)
hSideBand_15bin.SetLineWidth(3)
hSideBand_15bin.SetMarkerSize(1)

hSideBand_88bin = r.TH1D("hSideBand_88bin","",88,600,5000)
hSideBand_88bin.SetMarkerColor(2)
hSideBand_88bin.SetLineColor(2)
hSideBand_88bin.SetLineWidth(3)
hSideBand_88bin.SetMarkerSize(1)

treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_4bin",cutString_Signal)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_15bin",cutString_Signal)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSignal_88bin",cutString_Signal)

treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSideBand_15bin",cutString_SideBand)
treeIn.Draw("mass_lvj_type0_PuppiAK8>>hSideBand_88bin",cutString_SideBand)

4812L

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


### Plot the Wjet histogram in signal region and side-band region

In [4]:
leg = r.TLegend(0.45,0.95,.95,0.75)

leg.AddEntry(hSignal_15bin,"signal region (MC)")
leg.AddEntry(hSideBand_15bin,"SideBand Region (MC)")

hSignal_15bin.SetStats(0)
hSideBand_15bin.SetStats(0)


c1 = r.TCanvas("c1", "canvas", 1000, 600)
c1.SetLogy()
c1.Divide(2,1)
c1.cd(1)
r.gPad.SetLogy()
hSignal_15bin.SetTitle("Log axis")
hSignal_15bin.Draw()
hSideBand_15bin.Draw("same")
leg.Draw()
c1.cd(2)
hSignal_15bin.SetTitle("Linear axis")
hSignal_15bin.Draw()
hSideBand_15bin.Draw("same")
leg.Draw()

c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# Part-I

## Calculate alpha 

> by dividing W+jets MC in signal and side-band region


> fit the alpha

In [5]:
try: 
    Alpha.Delete();
except: 
    print "Alpha does not exists";

# add overflow bin

hSignal_15bin.SetBinContent(15,hSignal_15bin.GetBinContent(15)+hSignal_15bin.GetBinContent(16))
hSideBand_15bin.SetBinContent(15,hSideBand_15bin.GetBinContent(15)+hSideBand_15bin.GetBinContent(16))

print "hSignal_15bin overflow bin = ",hSignal_15bin.GetBinContent(16)
print "hSideBand_15bin overflow bin = ",hSideBand_15bin.GetBinContent(16)

    
Alpha =  r.TH1D("Alpha","",15,600,5000)
Alpha = hSignal_15bin.Clone()
Alpha.Divide(hSideBand_15bin)
c1 = r.TCanvas("c1", "canvas", 800, 600)
Alpha.GetXaxis().SetTitle("mWW (GeV)")
Alpha.GetYaxis().SetTitle("alpha (Signal/Side-band)")
Alpha.SetMaximum(4.0)
Alpha.SetMinimum(-1.0)
Alpha.Draw()
f1 = r.TF1("f1","pol1",600,5000)
frp = Alpha.Fit("f1","S")
c1.Draw()
print "Integral = ",Alpha.Integral("width")
print "Integral = ",f1.Integral(600,5000)
#for i in range(1,39):
#    print i,",",Alpha.GetBinContent(i),",",Alpha.GetBinLowEdge(i),",",Alpha.GetBinWidth(i)

print "chi2 = ",f1.GetChisquare()
print "NDF = ",f1.GetNDF()
print "par0 = ",f1.GetParameter(0)
print "par1 = ",f1.GetParameter(1)

Alpha does not exists
hSignal_15bin overflow bin =  0.0
hSideBand_15bin overflow bin =  0.00118848617931

****************************************
Minimizer is Linear
Chi2                      =      11.3722
NDf                       =            9
p0                        =       1.2163   +/-   0.0764802   
p1                        = -0.000342417   +/-   5.71463e-05 
Integral =  3940.23807613
Integral =  1133.14687026
chi2 =  11.372244129
NDF =  9
par0 =  1.21630065482
par1 =  -0.000342416884004


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [6]:
c1 = r.TCanvas("c1", "canvas", 1000, 900)

leg = r.TLegend(0.16,0.95,.65,0.75)
leg.SetNColumns(2);

hConf1s = Alpha.Clone("hConf1s");
hConf2s = Alpha.Clone("hConf2s");
hConf1s.Reset();
hConf2s.Reset();

r.TVirtualFitter.GetFitter().GetConfidenceIntervals(hConf1s, 0.68);
r.TVirtualFitter.GetFitter().GetConfidenceIntervals(hConf2s, 0.95);
hConf1s.SetStats(0);
hConf2s.SetStats(0);
hConf1s.SetMarkerSize(0);
hConf2s.SetMarkerSize(0);


hConf1s.SetFillColor(r.kGreen+1);
hConf2s.SetFillColor(r.kOrange);


hConf1s.SetLineColor(r.kGreen+1);
hConf2s.SetLineColor(r.kOrange);

hConf2s.SetTitle("")
hConf2s.SetMaximum(2.5)
hConf2s.Draw("e3");
hConf1s.Draw("e3 same");

Alpha.Draw("E1 same");
f1.Draw("same")
leg.AddEntry(Alpha,"MC","l")
leg.AddEntry(f1,"Fit","l")


leg.AddEntry(hConf1s,"1 #sigma band")
leg.AddEntry(hConf2s,"2 #sigma band")

leg.Draw()

print "chi2 = ",f1.GetChisquare()
print "NDF = ",f1.GetNDF()
print "par0 = ",f1.GetParameter(0)
print "par1 = ",f1.GetParameter(1)

pt = r.TPaveText(.65,.75,.98,.95,"brNDC");
pt.AddText("Chi2/ndf = "+str(round(f1.GetChisquare(),2))+"/"+str(f1.GetNDF()))
pt.AddText("a = "+str(round(f1.GetParameter(0),2))+" +/- "+str(round(f1.GetParError(0),2)))
pt.AddText("b = "+str(round(f1.GetParameter(1),4))+" +/- "+str(round(f1.GetParError(1),7)))

pt.Draw()

c1.Draw()

chi2 =  11.372244129
NDF =  9
par0 =  1.21630065482
par1 =  -0.000342416884004


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [7]:
numberOfPars = f1.GetNpar();
print numberOfPars

initialPars = []
initialParErrors = []
for ipar in xrange(numberOfPars):
    par = f1.GetParameter(ipar)
    parerror = f1.GetParError(ipar)
    initialPars.append(par)
    initialParErrors.append(parerror)
    print par,parerror

2
1.21630065482 0.0764801612716
-0.000342416884004 5.71463043008e-05


In [8]:
cov = frp.GetCovarianceMatrix();  #  to access the covariance matrix
cov.Print("V")


2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |   0.005849   -3.88e-06 
   1 |  -3.88e-06   3.266e-09 



In [9]:
# Find the EigenValue and EigenVector
EigenCov = r.TMatrixDSymEigen(cov)

#EigenCov.Print("V")

In [10]:
EigenVector_matrix = EigenCov.GetEigenVectors()
EigenValues = EigenCov.GetEigenValues()

EigenVector_matrix.Print()
EigenValues.Print()


2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |          1  -0.0006633 
   1 | -0.0006633          -1 


Vector (2)  is as follows

     |        1  |
------------------
   0 |0.00584922 
   1 |6.91989e-10 



In [11]:
eigenvectors =[]
for i in xrange(EigenVector_matrix.GetNrows()):
    eigenvector = r.TVectorD(EigenVector_matrix.GetNrows())
    for j in xrange(EigenVector_matrix.GetNrows()):
        eigenvector[j] = EigenVector_matrix[j][i]
    #eigenvector.Print()
    eigenvectors.append(eigenvector)

In [12]:
systFunctions = []
names = []

eigenvector = r.TVectorD()

f1UpArr = []
f1DownArr = []

# f1Up = []
# f1Down = []

#c1.Clear()

color = [600,820, 400,840,920, 616,860,632, 432,880,416, 800,900,910, 620,602,434, 1]

for k in xrange(EigenVector_matrix.GetNrows()):
        eigenvector = eigenvectors[k]
        norm = eigenvector.Norm2Sqr()
        eigenvalue = EigenValues[k]
        sigma = math.sqrt(r.TMath.Abs(eigenvalue))

        # compute unit vector in direction of i-th Eigenvector
        # eigenvector_unit = (1.0/float(norm))*eigenvector
        eigenvector_unit = eigenvector
        upPars = []
        downPars = []
        
        #print "initial pars = ",len(initialPars)
        print "DEBUG: 1: ",k,"=="*10
        #print "eigenvector:"
        #eigenvector.Print()
        #print "Norm = ",norm
        #print "Eigenvalue : ",eigenvalue
        #eigenvalue.Print()
        #print "=="*10
        #print "sigma = ",sigma
        for i in xrange(len(initialPars)):
            newParUp = initialPars[i] + sigma*eigenvector_unit[i]
            #print "initialPars[",i,"] = ",initialPars[i],"\t sigma = ",sigma,"\teigenvector_unit = ",eigenvector_unit[i]
            upPars.append(newParUp)
            print "==> UP: ",initialPars[i],"\t",newParUp,"\t",sigma,"\t",eigenvector_unit[i]
        for i in xrange(len(initialPars)):
            newParDown = initialPars[i] - sigma*eigenvector_unit[i]
            #print "initialPars[",i,"] = ",initialPars[i],"\t sigma = ",sigma,"\teigenvector_unit = ",eigenvector_unit[i]
            downPars.append(newParDown)
            print "==> Down: ",initialPars[i],"\t",newParDown,"\t",sigma,"\t",eigenvector_unit[i]
        print "--"*21
        print "Up pars..."
        print "--"*21
        
        fupName = "F_up_"+str(k)
        fdownName = "F_down_"+str(k)
        print "Fup name = ",fupName
        f1Up = r.TF1(fupName,"pol1",600,5000)
        f1Up.SetLineColor(color[k*2])
        print "print :",f1Up
        f1Down = r.TF1(fdownName,"pol1",600,5000)
        f1Down.SetLineColor(k*2+1)
        print "print :",f1Down

        for i in xrange(len(initialPars)):
            f1Up.FixParameter(i,upPars[i])
            #f1UpArr.append(f1Up)
            #print upPars[i]
        #f1Up.Delete()
        f1UpArr.append(f1Up)
        #print f1UpArr
        
        for i in xrange(len(initialPars)):
            f1Down.FixParameter(i,downPars[i])
            #f1UpArr.append(f1Up)
            #print downPars[i]
        #f1Up.Delete()
        f1DownArr.append(f1Down)
            
for k in xrange(EigenVector_matrix.GetNrows()):
    print "\n\n======================\n\n"
    print "Type of fun : ",type(f1UpArr[k])
    print "print :",f1UpArr[k]
    print "Type of fun : ",type(f1DownArr[k])
    print "print :",f1DownArr[k]
    f1UpArr[k].Draw("same")    
    f1DownArr[k].Draw("same")
    leg.AddEntry(f1UpArr[k],f1UpArr[k].GetName(),"l")
    leg.AddEntry(f1DownArr[k],f1DownArr[k].GetName(),"l")


    
leg.Draw()
c1.Draw()

DEBUG: 1:  0 ====================
==> UP:  1.21630065482 	1.29278081609 	0.0764801780976 	0.999999779995
==> UP:  -0.000342416884004 	-0.000393148643968 	0.0764801780976 	-0.000663332136848
==> Down:  1.21630065482 	1.13982049354 	0.0764801780976 	0.999999779995
==> Down:  -0.000342416884004 	-0.00029168512404 	0.0764801780976 	-0.000663332136848
------------------------------------------
Up pars...
------------------------------------------
Fup name =  F_up_0
print : <ROOT.TF1 object ("F_up_0") at 0x6e28a40>
print : <ROOT.TF1 object ("F_down_0") at 0x6e3a3e0>
DEBUG: 1:  1 ====================
==> UP:  1.21630065482 	1.21630063737 	2.63056824602e-05 	-0.000663332136848
==> UP:  -0.000342416884004 	-0.000368722560677 	2.63056824602e-05 	-0.999999779995
==> Down:  1.21630065482 	1.21630067227 	2.63056824602e-05 	-0.000663332136848
==> Down:  -0.000342416884004 	-0.000316111207331 	2.63056824602e-05 	-0.999999779995
------------------------------------------
Up pars...
-------------------

In [13]:
hConf1s.Print("V")

TH1.Print Name  = hConf1s, Entries= 15, Total sum= 3.863


## Convert TF1 of alpha to histogram of 88 bins

In [14]:
leg = r.TLegend(0.2,0.95,.70,0.75)
leg.SetNColumns(2);

hAlpha_Fit_88bin = r.TH1F("hAlpha_Fit_88bin","",88,600,5000)
count = 1
import numpy as np
print "Print alpha values in each bin: \n\n"
for i in np.arange(600,5000,50):
    print count,"\t",i,i+50,"=>",(i+i+50)/2.,f1.Eval((i+i+50)/2.)
    hAlpha_Fit_88bin.SetBinContent(count,f1.Eval((i+i+50)/2.));
    count+=1

hAlpha_Fit_88bin.SetMarkerColor(1)
hAlpha_Fit_88bin.SetLineColor(1)
hAlpha_Fit_88bin.SetLineWidth(3)
hAlpha_Fit_88bin.SetMarkerSize(1)


leg.AddEntry(hAlpha_Fit_88bin,"Nominal Alpha")
hAlpha_Fit_88bin.Draw()    
    
leg.Draw()

c1.Update()
c1.Draw()

Print alpha values in each bin: 


1 	600 650 => 625.0 1.00229010231
2 	650 700 => 675.0 0.985169258113
3 	700 750 => 725.0 0.968048413913
4 	750 800 => 775.0 0.950927569713
5 	800 850 => 825.0 0.933806725513
6 	850 900 => 875.0 0.916685881312
7 	900 950 => 925.0 0.899565037112
8 	950 1000 => 975.0 0.882444192912
9 	1000 1050 => 1025.0 0.865323348712
10 	1050 1100 => 1075.0 0.848202504512
11 	1100 1150 => 1125.0 0.831081660311
12 	1150 1200 => 1175.0 0.813960816111
13 	1200 1250 => 1225.0 0.796839971911
14 	1250 1300 => 1275.0 0.779719127711
15 	1300 1350 => 1325.0 0.762598283511
16 	1350 1400 => 1375.0 0.74547743931
17 	1400 1450 => 1425.0 0.72835659511
18 	1450 1500 => 1475.0 0.71123575091
19 	1500 1550 => 1525.0 0.69411490671
20 	1550 1600 => 1575.0 0.67699406251
21 	1600 1650 => 1625.0 0.659873218309
22 	1650 1700 => 1675.0 0.642752374109
23 	1700 1750 => 1725.0 0.625631529909
24 	1750 1800 => 1775.0 0.608510685709
25 	1800 1850 => 1825.0 0.591389841509
26 	1850 1900 => 1875.0 0.57

In [15]:
leg = r.TLegend(0.18,0.95,.65,0.75)
leg.SetNColumns(2);


hists = []
hists.append(hAlpha_Fit_88bin)
for k in xrange(EigenVector_matrix.GetNrows()):
    histUpName = "Corrected_Vjets_Up_"+str(k)
    histDownName = "Corrected_Vjets_Down_"+str(k)
    histUp   = r.TH1F(histUpName,"",88,600,5000)
    histDown = r.TH1F(histDownName,"",88,600,5000)


    print "==> ",color[k*2],"\t",color[k*2+1]    


    histUp.SetMarkerColor(color[k*2])
    histUp.SetLineColor(color[k*2])
    #histUp.SetFillColor(1)
    histUp.SetLineWidth(3)
    histUp.SetMarkerSize(1)
    #leg.AddEntry(histUp,histUp.GetName())

    histDown.SetMarkerColor(color[k*2+1])
    histDown.SetLineColor(color[k*2+1])
    #histDown.SetFillColor(1)
    histDown.SetLineWidth(3)
    histDown.SetMarkerSize(1)
    #leg.AddEntry(histDown,histDown.GetName())
    
    count=1
    #print "fun up/down val"

    for i in np.arange(600,5000,50):
        histUp.SetBinContent(count,f1UpArr[k].Eval((i+i+50)/2.));
        histDown.SetBinContent(count,f1DownArr[k].Eval((i+i+50)/2.));
        #print i,"\t",f1UpArr[k].Eval((i+i+50)/2.),f1DownArr[k].Eval((i+i+50)/2.)       
        count+=1

    histUp.SetStats(0)
    histDown.SetStats(0)
    hists.append(histUp)
    hists.append(histDown)


for k in xrange(len(hists)):
    if k==0:
        leg.AddEntry(hists[k],"Nominal Alpha")
        hists[k].SetStats(0)
        hists[k].Draw()
    else:
        leg.AddEntry(hists[k],hists[k].GetName(),"l")
        hists[k].SetStats(0)
        hists[k].Draw("same")

hists[0].SetMaximum(2.0)
Alpha.Draw("E1 same");
leg.Draw()

print "chi2 = ",f1.GetChisquare()
print "NDF = ",f1.GetNDF()
print "par0 = ",f1.GetParameter(0)
print "par1 = ",f1.GetParameter(1)

pt = r.TPaveText(.65,.75,.98,.95,"brNDC");
pt.AddText("Chi2/ndf = "+str(round(f1.GetChisquare(),2))+"/"+str(f1.GetNDF()))
pt.AddText("a = "+str(round(f1.GetParameter(0),2))+" +/- "+str(round(f1.GetParError(0),2)))
pt.AddText("b = "+str(round(f1.GetParameter(1),4))+" +/- "+str(round(f1.GetParError(1),7)))

pt.Draw()
c1.Update()
c1.Draw()
c1.SaveAs("WVchannel_AlphaDistribution_AfterFit.png")
c1.SaveAs("WVchannel_AlphaDistribution_AfterFit.pdf")

==>  600 	820
==>  400 	840
chi2 =  11.372244129
NDF =  9
par0 =  1.21630065482
par1 =  -0.000342416884004


Info in <TCanvas::Print>: png file WVchannel_AlphaDistribution_AfterFit.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_AlphaDistribution_AfterFit.pdf has been created


In [16]:
c1.SetLogy(0)
c1.Draw()
c1.SaveAs("WVchannel_AlphaDistribution_AfterFit.png")
c1.SaveAs("WVchannel_AlphaDistribution_AfterFit.pdf")

Info in <TCanvas::Print>: png file WVchannel_AlphaDistribution_AfterFit.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_AlphaDistribution_AfterFit.pdf has been created


# Get corrected W+jet in sideband region after fit to data (From Background Estimation Code)

This histogram is saved in ExoVV fitter using create histogram method:

https://github.com/ram1123/EXOVVFitter/blob/master/g1_exo_doFit_class_new.py#L1795-L1796

But, this histogram was not taking the normalization so I have to scale it with the normalization. ***But, scaling the histogram also scales the errors.***

## Check normalization in sideband region

In [17]:
!grep -A 10 "_WJets01_xww+++" WWTree_CommonNtuple_For1and2Lepton_2018_05_15_04h15_WV_600_5TeV/cards_em_HP/other_wwlvj_Signal_aQGC600_em_HP.txt  | grep "Events Number in sideband_low from fitting\|Events Number in sideband_high from fitting"

Events Number in sideband_low from fitting:145.362603475 +/- 34.2269073605
Events Number in sideband_high from fitting:95.2938212854 +/- 34.2269073605


In [18]:
print "Normalization = ",145.362603475+95.2988212854

Normalization =  240.66142476


In [19]:
leg = r.TLegend(0.48,0.95,.95,0.75)


c1.Clear()
r.gStyle.SetOptStat(0)

Wjet_file = r.TFile.Open("WWTree_CommonNtuple_For1and2Lepton_2018_05_15_04h15_WV_600_5TeV/wjetmodel_Ex__WJets0_xww__sb_lo_ExpTail_auto.root")

Wjet_hist = r.TH1F()
Wjet_hist = Wjet_file.Get("rrv_mass_lvj__rrv_mass_lvj")
Wjet_hist.Scale(240.65642476)
print Wjet_hist.Integral()
print "Number of bins = ",Wjet_hist.GetSize()
print "Overflow bin = ",Wjet_hist.GetBinContent(89)
#Wjet_hist.SetMaximum(150)

Wjet_hist.SetStats(0)
hSideBand_88bin.SetStats(0)


Wjet_hist.Draw("hist")
hSideBand_88bin.Draw("same")
leg.AddEntry(Wjet_hist,"SideBand Region (Corr)")
leg.AddEntry(hSideBand_88bin,"SideBand Region (MC)")
leg.Draw()
c1.SetLogy()
w# WVchannel_Comparison_VjetShape_MC_CorrShapeFromData.png
c1.SaveAs("WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData_log.png")
c1.SaveAs("WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData_log.pdf")

240.301325727
Number of bins =  90
Overflow bin =  0.0


Info in <TCanvas::Print>: png file WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData_log.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData_log.pdf has been created


In [20]:
c1.SetLogy(0)
c1.Draw()
c1.SaveAs("WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData.png")
c1.SaveAs("WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData.pdf")

Info in <TCanvas::Print>: png file WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_SideBandRegionComparison_VjetShape_MC_CorrShapeFromData.pdf has been created


# Multiplication by Alpha to get Signal Region W+jets obtained from data.

In [21]:
leg = r.TLegend(0.4,0.95,.95,0.75)
leg.SetNColumns(2);

histSig = []

for k in xrange(len(hists)):
    hModel_Sig = r.TH1F(hists[k].GetName(),"",88,600,5000)
    hModel_Sig.Sumw2()
    hModel_Sig.SetMarkerColor(color[k])
    hModel_Sig.SetLineColor(color[k])
    hModel_Sig.SetLineWidth(3)
    hModel_Sig.SetMarkerSize(1)
    hModel_Sig.SetStats(0)
    
    for i in range(1,39):
        hModel_Sig.SetBinContent(i,Wjet_hist.GetBinContent(i)*hists[k].GetBinContent(i))
        hModel_Sig.SetBinError(i,0.0)


    histSig.append(hModel_Sig)
    
for k in xrange(len(histSig)):
    if k==0:
        leg.AddEntry(histSig[k],"Corrected_Vjets_Nominal")
        histSig[k].Draw()
    else:
        leg.AddEntry(histSig[k],hists[k].GetName())
        histSig[k].Draw("same")

histSig[0].SetMaximum(55.0)
hSignal_88bin.Draw("E1 same")
leg.AddEntry(hSignal_88bin,"MC")
leg.Draw()
c1.Draw()
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin.png")
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin.pdf")

Info in <TCanvas::Print>: png file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin.pdf has been created


In [22]:
c1.SetLogy(1)
c1.Draw()
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin_log.png")
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin_log.pdf")

Info in <TCanvas::Print>: png file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin_log.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_88bin_log.pdf has been created


In [23]:
leg = r.TLegend(0.4,0.95,.90,0.75)
leg.SetNColumns(2);

histSig_4bin = []

for k in xrange(len(hists)):
    hModel_Sig_4bin = r.TH1F(hists[k].GetName(),"",4,600,2500)
    hModel_Sig_4bin.Sumw2()
    hModel_Sig_4bin.SetMarkerColor(color[k])
    hModel_Sig_4bin.SetLineColor(color[k])
    hModel_Sig_4bin.SetLineWidth(3)
    hModel_Sig_4bin.SetMarkerSize(1)
    hModel_Sig_4bin.SetStats(0)
    
    
    count = 1
    bin = [0]*5
    # the if conditions are defined manually to make 4 bins exactly same as 2.5 TeV scale.
    for i in range(1,89):
        if i > 28:
            bin[4] += histSig[k].GetBinContent(i)
        elif i > 19:
            bin[3] += histSig[k].GetBinContent(i)
        elif i > 9:
            bin[2] += histSig[k].GetBinContent(i)
        else:
            bin[1] += histSig[k].GetBinContent(i)

    hModel_Sig_4bin.SetBinContent(1,bin[1])
    hModel_Sig_4bin.SetBinContent(2,bin[2])
    hModel_Sig_4bin.SetBinContent(3,bin[3])
    hModel_Sig_4bin.SetBinContent(4,bin[4])
    print "bin 1",hModel_Sig_4bin.GetBinContent(1),"\t Range (",hModel_Sig_4bin.GetBinLowEdge(1),",",hModel_Sig_4bin.GetBinLowEdge(2),")"
    print "bin 2",hModel_Sig_4bin.GetBinContent(2),"\t Range (",hModel_Sig_4bin.GetBinLowEdge(2),",",hModel_Sig_4bin.GetBinLowEdge(3),")"
    print "bin 3",hModel_Sig_4bin.GetBinContent(3),"\t Range (",hModel_Sig_4bin.GetBinLowEdge(3),",",hModel_Sig_4bin.GetBinLowEdge(4),")"
    print "bin 4",hModel_Sig_4bin.GetBinContent(4),"\t Range (",hModel_Sig_4bin.GetBinLowEdge(4),",",hModel_Sig_4bin.GetBinLowEdge(5),")"


    hModel_Sig_4bin.SetBinError(1,0.0)
    hModel_Sig_4bin.SetBinError(2,0.0)
    hModel_Sig_4bin.SetBinError(3,0.0)
    hModel_Sig_4bin.SetBinError(4,0.0)
    
    histSig_4bin.append(hModel_Sig_4bin)
    
for k in xrange(len(histSig_4bin)):
    if k==0:
        leg.AddEntry(histSig_4bin[k],"Nominal Sig")
        histSig_4bin[k].Draw()
        histSig_4bin[k].SaveAs("WVchannel_Hist_Nominal_Sig_4bins.root")
    else:
        leg.AddEntry(histSig_4bin[k],hists[k].GetName())
        histSig_4bin[k].Draw("same")
        histSig_4bin[k].SaveAs("WVchannel_"+hists[k].GetName()+"_4bins.root")

leg.Draw()
c1.Draw()
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin_log.png")
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin_log.pdf")

bin 1 183.265869141 	 Range ( 600.0 , 1075.0 )
bin 2 31.0688438416 	 Range ( 1075.0 , 1550.0 )
bin 3 4.30972766876 	 Range ( 1550.0 , 2025.0 )
bin 4 0.805949211121 	 Range ( 2025.0 , 2500.0 )
bin 1 190.495544434 	 Range ( 600.0 , 1075.0 )
bin 2 31.616355896 	 Range ( 1075.0 , 1550.0 )
bin 3 4.23469400406 	 Range ( 1550.0 , 2025.0 )
bin 4 0.746221661568 	 Range ( 2025.0 , 2500.0 )
bin 1 176.036178589 	 Range ( 600.0 , 1075.0 )
bin 2 30.5213317871 	 Range ( 1075.0 , 1550.0 )
bin 3 4.3847618103 	 Range ( 1550.0 , 2025.0 )
bin 4 0.865676701069 	 Range ( 2025.0 , 2500.0 )
bin 1 179.397644043 	 Range ( 600.0 , 1075.0 )
bin 2 29.8021736145 	 Range ( 1075.0 , 1550.0 )
bin 3 3.99800825119 	 Range ( 1550.0 , 2025.0 )
bin 4 0.706490337849 	 Range ( 2025.0 , 2500.0 )
bin 1 187.134078979 	 Range ( 600.0 , 1075.0 )
bin 2 32.3355140686 	 Range ( 1075.0 , 1550.0 )
bin 3 4.62144756317 	 Range ( 1550.0 , 2025.0 )
bin 4 0.905408084393 	 Range ( 2025.0 , 2500.0 )


Warning in <TFile::Append>: Replacing existing TH1: hAlpha_Fit_88bin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Corrected_Vjets_Up_0 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Corrected_Vjets_Down_0 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Corrected_Vjets_Up_1 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Corrected_Vjets_Down_1 (Potential memory leak).
Info in <TH1F::SaveAs>: ROOT file WVchannel_Hist_Nominal_Sig_4bins.root has been created
Info in <TH1F::SaveAs>: ROOT file WVchannel_Corrected_Vjets_Up_0_4bins.root has been created
Info in <TH1F::SaveAs>: ROOT file WVchannel_Corrected_Vjets_Down_0_4bins.root has been created
Info in <TH1F::SaveAs>: ROOT file WVchannel_Corrected_Vjets_Up_1_4bins.root has been created
Info in <TH1F::SaveAs>: ROOT file WVchannel_Corrected_Vjets_Down_1_4bins.root has been created
Info in <TCanvas::Print>: png file WVchannel_

In [24]:
c1.SetLogy(0)
c1.Draw()
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin.png")
c1.SaveAs("WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin.pdf")

Info in <TCanvas::Print>: png file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_SignalRegionComparison_VjetShape_MC_CorrShapeFromData_4bin.pdf has been created


In [25]:
c1.Clear()
fin_Wjet_MC = r.TFile.Open("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_VJets.root")
fin_VV      = r.TFile.Open("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_VV.root")
fin_TTbar   = r.TFile.Open("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_TTbar.root")
fin_STop    = r.TFile.Open("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_STop.root")
fin_Data    = r.TFile.Open("/eos/uscms/store/user/rasharma/SecondStep/WWTree_CommonNtuple_For1and2Lepton_2018_04_06_09h22/HaddedFiles/Hadds_for_BkgEstimation/WWTree_data_golden.root")

treeIn_Wjet_MC = fin_Wjet_MC.Get("otree")
treeIn_VV = fin_VV.Get("otree")
treeIn_TTbar = fin_TTbar.Get("otree")
treeIn_STop = fin_STop.Get("otree")
treeIn_Data = fin_Data.Get("otree")

# Data/MC with 88 bins

In [26]:
# For 88 bins
histSig[0].SetMarkerColor(r.TColor.GetColor(222,90,106))
histSig[0].SetLineColor(r.TColor.GetColor(222,90,106))
histSig[0].SetFillColor(r.TColor.GetColor(222,90,106))
histSig[0].SetLineWidth(3)
histSig[0].SetMarkerSize(1)

hWjets_MC = r.TH1D("hWjets_MC","",88,600,5000)
hWjets_MC.SetMarkerColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetLineColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetFillColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetLineWidth(3)
hWjets_MC.SetMarkerSize(1)

hVV = r.TH1D("hVV","",88,600,5000)
hVV.SetMarkerColor(r.TColor.GetColor(250,202,255))
hVV.SetLineColor(r.TColor.GetColor(250,202,255))
hVV.SetFillColor(r.TColor.GetColor(250,202,255))
hVV.SetLineWidth(3)
hVV.SetMarkerSize(1)

hTTbar = r.TH1D("hTTbar","",88,600,5000)
hTTbar.SetMarkerColor(r.TColor.GetColor(155,152,204))
hTTbar.SetLineColor(r.TColor.GetColor(155,152,204))
hTTbar.SetFillColor(r.TColor.GetColor(155,152,204))
hTTbar.SetLineWidth(3)
hTTbar.SetMarkerSize(1)

hSTop = r.TH1D("hSTop","",88,600,5000)
hSTop.SetMarkerColor(r.TColor.GetColor(0,84,159))
hSTop.SetLineColor(r.TColor.GetColor(0,84,159))
hSTop.SetFillColor(r.TColor.GetColor(0,84,159))
hSTop.SetLineWidth(3)
hSTop.SetMarkerSize(1)

hData = r.TH1D("hData","",88,600,5000)
hData.SetMarkerColor(1)
hData.SetLineColor(1)
#hData.SetFillColor(1)
hData.SetLineWidth(3)
hData.SetMarkerSize(1)

treeIn_Wjet_MC.Draw("mass_lvj_type0_PuppiAK8>>hWjets_MC",cutString_SideBand)
treeIn_VV.Draw("mass_lvj_type0_PuppiAK8>>hVV",cutString_SideBand)
treeIn_TTbar.Draw("mass_lvj_type0_PuppiAK8>>hTTbar",cutString_SideBand)
treeIn_STop.Draw("mass_lvj_type0_PuppiAK8>>hSTop",cutString_SideBand)

#cutString_SigData =  "((l_pt2<0) && ((l_pt1>30.) && (abs(l_eta1)<2.5)) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) &&  (nBTagJet_loose==0) && (pfMET_Corr>50) && ((PuppiAK8_jet_tau2tau1<0.55)) && ((PuppiAK8_jet_mass_so_corr>55) && (PuppiAK8_jet_mass_so_corr<65)) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && (vbf_maxpt_jj_m>800) && (BosonCentrality_type0>1.0) && (abs(ZeppenfeldWL_type0/vbf_maxpt_jj_Deta)<0.3) && (abs(ZeppenfeldWH/vbf_maxpt_jj_Deta)<0.3) && (mass_lvj_type0>170))"

cutString_SigData_Data =  "((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800 && vbf_maxpt_jj_m>800 ) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && ((PuppiAK8_jet_mass_so_corr>65) && (PuppiAK8_jet_mass_so_corr<105)) && (mass_lvj_type0_PuppiAK8>600 && mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

cutString_SideBand_Data =  "((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800 && vbf_maxpt_jj_m>800 ) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && (((PuppiAK8_jet_mass_so_corr>40)&&(PuppiAK8_jet_mass_so_corr<65)) || ((PuppiAK8_jet_mass_so_corr>105)&&(PuppiAK8_jet_mass_so_corr<150))) && (mass_lvj_type0_PuppiAK8>600 && mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

cutString_Lower_SideBand_Data =  "((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800 && vbf_maxpt_jj_m>800 ) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && (((PuppiAK8_jet_mass_so_corr>40)&&(PuppiAK8_jet_mass_so_corr<65))) && (mass_lvj_type0_PuppiAK8>600 && mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"




treeIn_Data.Draw("mass_lvj_type0_PuppiAK8>>hData",cutString_SideBand_Data)

343L

In [27]:
leg = r.TLegend(0.55,0.95,.90,0.72)

stack = r.THStack()

stack.Add(hSTop,"F")
stack.Add(hTTbar,"F")
stack.Add(hVV,"F")
stack.Add(histSig[0],"F")
#stack.Add(hModel_Sig_new,"F")

leg.AddEntry(hSTop,"STop")
leg.AddEntry(hTTbar,"TTbar")
leg.AddEntry(hVV,"Diboson")
leg.AddEntry(histSig[0],"Vjets")
#leg.AddEntry(hModel_Sig_new,"Wjets")
leg.AddEntry(hData,"Data")



#stack.SetMinimum(0.001)
stack.SetMaximum(90.0)
#stack.SetMaximum(400.0)

#stack.Draw("hist")
#hData.Draw("same hist")
#leg.Draw()
#c1.Draw()

In [28]:
c1 = r.TCanvas("c1", "canvas", 800, 600)
c1.SetLogy(0)
CMS_lumi.lumi_13TeV = "35.9 fb^{-1}"
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "Preliminary"

CMS_lumi.relPosX = 0.05

rp = r.TRatioPlot(stack, hData);
#c.SetTicks(0, 1);
rp.Draw();
rp.GetLowerRefYaxis().SetTitle("MC/Data");
rp.GetUpperRefYaxis().SetTitle("Events/(50 GeV)");
rp.GetLowerRefXaxis().SetTitle("M_{ww}");
rp.GetLowerRefGraph().SetMinimum(0.1);
rp.GetLowerRefGraph().SetMaximum(2.8);

rp.SetLeftMargin(0.14);
# rp.SetRightMargin(0.05);
# rp.SetUpTopMargin(0.05);
# rp.SetSeparationMargin(0.005);
# rp.SetLowBottomMargin(0.46);

# rp.Draw("errasymfhidelow");
# rp.SetSplitFraction(0.10);
# rp.GetLowYaxis().SetNdivisions(4);
# rp.GetLowerRefYaxis().SetTitle("Pred./Data");
# rp.GetLowerRefGraph().SetMinimum(1.5);
# rp.GetLowerRefGraph().SetMaximum(0.5);
# rp.GetLowerRefYaxis().SetRangeUser(0.5, 1.5);
    
    
rp.SetSeparationMargin(0.0)
leg.Draw()

CMS_lumi.CMS_lumi(c1, 4, 11)
c1.Update();
c1.Draw()

c1.SaveAs("WVchannel_DataMC-Comparison_AfterCorrVjetShape_88bin.png")
c1.SaveAs("WVchannel_DataMC-Comparison_AfterCorrVjetShape_88bin.pdf")

35.9 fb^{-1} (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 66 points have been skipped
Info in <TCanvas::Print>: png file WVchannel_DataMC-Comparison_AfterCorrVjetShape_88bin.png has been created
Info in <TCanvas::Print>: pdf file WVchannel_DataMC-Comparison_AfterCorrVjetShape_88bin.pdf has been created


# Data/MC with 4 bins

In [29]:
c1.Clear()
fin_Wjet_MC = r.TFile.Open("Ntuples2/WWTree_VJets.root")
fin_VV      = r.TFile.Open("Ntuples2/WWTree_VV.root")
fin_TTbar   = r.TFile.Open("Ntuples2/WWTree_TTbar.root")
fin_STop    = r.TFile.Open("Ntuples2/WWTree_STop.root")
fin_Data    = r.TFile.Open("Ntuples2/WWTree_data_golden.root")

treeIn_Wjet_MC = fin_Wjet_MC.Get("otree")
treeIn_VV = fin_VV.Get("otree")
treeIn_TTbar = fin_TTbar.Get("otree")
treeIn_STop = fin_STop.Get("otree")
treeIn_Data = fin_Data.Get("otree")

In [30]:
histSig_4bin[0].SetMarkerColor(r.TColor.GetColor(222,90,106))
histSig_4bin[0].SetLineColor(r.TColor.GetColor(222,90,106))
histSig_4bin[0].SetFillColor(r.TColor.GetColor(222,90,106))
histSig_4bin[0].SetLineWidth(3)
histSig_4bin[0].SetMarkerSize(1)

hWjets_MC = r.TH1D("hWjets_MC","",4,600,5000)
hWjets_MC.SetMarkerColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetLineColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetFillColor(r.TColor.GetColor(250,202,255))
hWjets_MC.SetLineWidth(3)
hWjets_MC.SetMarkerSize(1)

hVV = r.TH1D("hVV","",4,600,5000)
hVV.SetMarkerColor(r.TColor.GetColor(250,202,255))
hVV.SetLineColor(r.TColor.GetColor(250,202,255))
hVV.SetFillColor(r.TColor.GetColor(250,202,255))
hVV.SetLineWidth(3)
hVV.SetMarkerSize(1)

hTTbar = r.TH1D("hTTbar","",4,600,5000)
hTTbar.SetMarkerColor(r.TColor.GetColor(155,152,204))
hTTbar.SetLineColor(r.TColor.GetColor(155,152,204))
hTTbar.SetFillColor(r.TColor.GetColor(155,152,204))
hTTbar.SetLineWidth(3)
hTTbar.SetMarkerSize(1)

hSTop = r.TH1D("hSTop","",4,600,5000)
hSTop.SetMarkerColor(r.TColor.GetColor(0,84,159))
hSTop.SetLineColor(r.TColor.GetColor(0,84,159))
hSTop.SetFillColor(r.TColor.GetColor(0,84,159))
hSTop.SetLineWidth(3)
hSTop.SetMarkerSize(1)

hData = r.TH1D("hData","",4,600,5000)
hData.SetMarkerColor(1)
hData.SetLineColor(1)
#hData.SetFillColor(1)
hData.SetLineWidth(3)
hData.SetMarkerSize(1)

treeIn_Wjet_MC.Draw("mass_lvj_type0_PuppiAK8>>hWjets_MC",cutString_SideBand)
treeIn_VV.Draw("mass_lvj_type0_PuppiAK8>>hVV",cutString_SideBand)
treeIn_TTbar.Draw("mass_lvj_type0_PuppiAK8>>hTTbar",cutString_SideBand)
treeIn_STop.Draw("mass_lvj_type0_PuppiAK8>>hSTop",cutString_SideBand)

#cutString_SigData =  "((l_pt2<0) && ((l_pt1>30.) && (abs(l_eta1)<2.5)) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) &&  (nBTagJet_loose==0) && (pfMET_Corr>50) && ((PuppiAK8_jet_tau2tau1<0.55)) && ((PuppiAK8_jet_mass_so_corr>55) && (PuppiAK8_jet_mass_so_corr<65)) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && (vbf_maxpt_jj_m>800) && (BosonCentrality_type0>1.0) && (abs(ZeppenfeldWL_type0/vbf_maxpt_jj_Deta)<0.3) && (abs(ZeppenfeldWH/vbf_maxpt_jj_Deta)<0.3) && (mass_lvj_type0>170))"

cutString_SigData_Data   =  "((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800 && vbf_maxpt_jj_m>800 ) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && ((PuppiAK8_jet_mass_so_corr>65) && (PuppiAK8_jet_mass_so_corr<105)) && (mass_lvj_type0_PuppiAK8>600 && mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"

cutString_SideBand_Data  =  "((type==0||type==1) && (l_pt2<0 && l_pt1>30 && (((type==0)&&(abs(l_eta1)<2.4)) || ((type==1)&&((abs(l_eta1)<2.5)&&!(abs(l_eta1)>1.4442 && abs(l_eta1)<1.566))))) && (((type==0)&&(pfMET_Corr>50)) || ((type==1)&&(pfMET_Corr>80))) && ((ungroomed_PuppiAK8_jet_pt>200)&&(abs(ungroomed_PuppiAK8_jet_eta)<2.4)&&(PuppiAK8_jet_tau2tau1<0.55)) && (nBTagJet_loose==0) && (vbf_maxpt_jj_m>800 && vbf_maxpt_jj_m>800 ) && (abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta)>4.0) && ((vbf_maxpt_j1_pt>30) && (vbf_maxpt_j2_pt>30)) && (((PuppiAK8_jet_mass_so_corr>40)&&(PuppiAK8_jet_mass_so_corr<65)) || ((PuppiAK8_jet_mass_so_corr>105)&&(PuppiAK8_jet_mass_so_corr<150))) && (mass_lvj_type0_PuppiAK8>600 && mass_lvj_type0_PuppiAK8>600) && ((abs(ZeppenfeldWL_type0)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && ((abs(ZeppenfeldWH)/abs(vbf_maxpt_j2_eta-vbf_maxpt_j1_eta))<0.3) && (BosonCentrality_type0>1.0))"



treeIn_Data.Draw("mass_lvj_type0_PuppiAK8>>hData",cutString_SideBand_Data)

321L

In [ ]:
leg.Clear();
stack = r.THStack()

stack.Add(hSTop,"F")
stack.Add(hTTbar,"F")
stack.Add(hVV,"F")
#stack.Add(hModel_Sig,"F")
stack.Add(histSig_4bin[0],"F")

leg.AddEntry(hSTop,"STop")
leg.AddEntry(hTTbar,"TTbar")
leg.AddEntry(hVV,"Diboson")
#leg.AddEntry(hModel_Sig,"Wjets")
leg.AddEntry(histSig_4bin[0],"Wjets")
leg.AddEntry(hData,"Data")

#stack.SetMinimum(0.001)
#stack.SetMaximum(90.0)
stack.SetMaximum(400.0)

stack.Draw("hist")
hData.Draw("same hist")
leg.Draw()


#c1.Draw()

In [ ]:
c1 = r.TCanvas("c1", "canvas", 800, 600)
c1.SetLogy(0)
CMS_lumi.lumi_13TeV = "35.9 fb^{-1}"
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "Preliminary"

CMS_lumi.relPosX = 0.05

rp = r.TRatioPlot(stack, hData);
#c.SetTicks(0, 1);
rp.Draw();
rp.GetLowerRefYaxis().SetTitle("MC/Data");
rp.GetUpperRefYaxis().SetTitle("Events/(50 GeV)");
rp.GetLowerRefXaxis().SetTitle("M_{ww}");
rp.GetLowerRefGraph().SetMinimum(0.1);
rp.GetLowerRefGraph().SetMaximum(2.8);

rp.SetLeftMargin(0.14);
    
    
rp.SetSeparationMargin(0.0)

CMS_lumi.CMS_lumi(c1, 4, 11)
c1.Update();
c1.Draw()
c1.SaveAs("WVchannel_DataMC-Comparison_AfterCorrVjetShape_4bin.png")
c1.SaveAs("WVchannel_DataMC-Comparison_AfterCorrVjetShape_4bin.pdf")

In [ ]:
c1.SetLogy(1)
c1.Draw()

In [ ]:
print "Program is done..."